In [62]:
!pip install pandas numpy matplotlib yfinance backtrader datetime timedelta
from IPython.core.display import clear_output
clear_output()

In [95]:
import yfinance as yf
import backtrader as bt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose

In [96]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

# Optionally, set the max rows displayed in the output as well
pd.set_option('display.max_rows', 100)

# Introduction
This notebook is designed to showcase a comprehensive analysis of a diverse portfolio, integrating Index Funds, Leveraged ETFs, monthly dividend REITs and ETFs, and quarterly dividend stocks. The analysis includes portfolio beta evaluation, backtesting a Dollar-Cost Averaging (DCA) strategy that optimizes timing and risk management with technical and quantative analysis, and hedged with options pricing.


In [97]:
# Define Portfolio Assets
index_funds = ['SPY', 'QQQ', 'DAX']
leveraged_etfs = ['TQQQ', 'UMDD', 'UDOW', 'SOXL', 'NVDL', 'TSLL']
monthly_dividend_reits_etfs = ['O', 'AGNC', 'CSHI', 'JEPI', 'NUSI']
quarterly_dividend_stocks = [
    'SPYD', 'MSFT', 'INTC', 'F', 'CSCO', 'BAC', 'PFE', 'BX', 'MO', 
    'DOW', 'WMT', 'T', 'KMB', 'SWK', 'IBM', 'PEP', 'KO', 'JNJ'
]

In [98]:
############################################################################################################
# Define Technical Indicators Functions
############################################################################################################
def bollinger_bands(data, window=20, num_std=2):
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    data['Bollinger_High'] = rolling_mean + (rolling_std * num_std)
    data['Bollinger_Low'] = rolling_mean - (rolling_std * num_std)
    return data

def macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['Signal'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

def rsi(data, periods=14, ema=True):
    close_delta = data['Close'].diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    if ema:
        ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
        ma_down = down.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
    else:
        ma_up = up.rolling(window=periods).mean()
        ma_down = down.rolling(window=periods).mean()
    rsi = ma_up / ma_down
    data['RSI'] = 100 - (100 / (1 + rsi))
    return data

def woodie_pivots(data):
    high = data['High']
    low = data['Low']
    close = data['Close']
    pivot = (high + low + 2 * close) / 4
    data['Pivot'] = pivot
    data['R1'] = 2 * pivot - low
    data['S1'] = 2 * pivot - high
    data['R2'] = pivot + (high - low)
    data['S2'] = pivot - (high - low)
    data['R3'] = high + 2 * (pivot - low)
    data['S3'] = low - 2 * (high - pivot)
    data['R4'] = pivot + 3 * (high - low)
    data['S4'] = pivot - 3 * (high - low)
    return data

def obv(data):
    data['OBV'] = np.where(data['Close'] > data['Close'].shift(1), data['Volume'],
                           np.where(data['Close'] < data['Close'].shift(1), -data['Volume'], 0)).cumsum()
    return data

def atr(data, window=14):
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift())
    low_close = np.abs(data['Low'] - data['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    data['ATR'] = true_range.rolling(window=window).mean()
    return data

def stochastic_oscillator(data, window=14):
    low_min = data['Low'].rolling(window=window).min()
    high_max = data['High'].rolling(window=window).max()
    data['%K'] = 100 * ((data['Close'] - low_min) / (high_max - low_min))
    data['%D'] = data['%K'].rolling(window=3).mean()
    return data

############################################################################################################
# Process other non-stationary data
############################################################################################################
# Function to detrend time series data using a linear regression model
def detrend_data(data, column):
    # Linear regression model requires reshaped index as a feature
    X = np.arange(len(data)).reshape(-1, 1)
    y = data[column].values  # Original values to detrend
    
    # Create and fit the model
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict the trend
    trend = model.predict(X)
    
    # Detrend by subtracting the trend from the original data
    detrended = y - trend
    data[f'{column}_detrended'] = detrended
    
    # Return the detrended data and the trend for further analysis
    return data, trend

def seasonal_decomposition(data, column, period):
    # Perform seasonal decomposition
    decomposition = seasonal_decompose(data[column], model='multiplicative', period=period)
    
    # Add components to DataFrame
    data['trend_component'] = decomposition.trend
    data['seasonal_component'] = decomposition.seasonal
    data['residual_component'] = decomposition.resid
    
    # Seasonally adjust the data
    data[column + '_seasonally_adjusted'] = data[column] / data['seasonal_component']
    
    return data

# Function to calculate price differences
def calculate_price_differences(data, column):
    data[f'{column}_diff'] = data[column].diff()
    return data

# Function to calculate log returns
def calculate_log_returns(data, column):
    data[f'{column}_log_return'] = np.log(data[column] / data[column].shift(1))
    return data

# Function to calculate volume changes
def calculate_volume_changes(data, volume_column):
    data[f'{volume_column}_changes'] = data[volume_column].diff()
    return data


In [99]:
############################################################################################################
# Fetch Financial Data Function with Technical Indicators
############################################################################################################
def fetch_financial_data(ticker='SPY', start_year=1993, end_year=None, interval='1d', export_csv=False, csv_file=None, calculate_indicators=False, include_fundamentals=False):
    """
    Fetches data for a specified ticker from Yahoo Finance from the given start year to the current year or specified end year at specified intervals.
    
    Parameters:
        ticker (str): The ticker symbol for the asset. Defaults to 'SPY'.
        start_year (int): The year from which to start fetching the data. Defaults to 1993.
        end_year (int): The last year for which to fetch the data. Defaults to the current year if None.
        interval (str): The data interval ('1d' for daily, '1wk' for weekly, '1mo' for monthly, '1h' for hourly).
        export_csv (bool): Whether to export the data to a CSV file. Defaults to False.
        csv_file (str): The path of the CSV file to export the data to. Automatically determined if None.
    """
    # Adjust for hourly data to limit to the last 730 days
    if interval == '1h':
        start_date = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')
    else:
        start_date = f'{start_year}-01-01'
    
    if end_year is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    else:
        end_date = f"{end_year}-12-31"
    
    if csv_file is None:
        csv_file = f'{ticker}_{interval}_data_{start_date}_to_{end_date}.csv'
    
    data = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)
    
    if not data.empty:
        if calculate_indicators:
            # Here you would call your indicator functions on the `data` DataFrame
            data = bollinger_bands(data)
            data = macd(data)
            data = rsi(data)
            data = woodie_pivots(data)
            data = obv(data)
            data = atr(data)
            data = stochastic_oscillator(data)

            # Non-stationary data processing
            data = calculate_price_differences(data, 'Close')  # Calculate price differences
            data = calculate_log_returns(data, 'Close')  # Calculate log returns for the 'Close' column
            data = calculate_volume_changes(data, 'Volume')  # Calculate volume changes
            # Handling NaN values by forward filling then dropping rows with NaN values
            data.ffill(inplace=True)
            data.dropna(inplace=True)

        if include_fundamentals:
            ticker_obj = yf.Ticker(ticker)
            try:
                # Attempt to fetch and include fundamental data directly in the data DataFrame
                # Note: Fundamental data is fetched separately and not integrated into the `data` DataFrame
                # due to differing data structures. Returned as part of a dictionary.
                fundamentals = {
                    'dividends': ticker_obj.dividends,
                    'splits': ticker_obj.splits,
                    'financials': ticker_obj.financials,
                    'balance_sheet': ticker_obj.balance_sheet,
                    'cashflow': ticker_obj.cashflow,
                    'info': ticker_obj.info  # Provides a dictionary of various fundamental data points
                }
            except Exception as e:
                print(f"Failed to fetch fundamental data for {ticker}: {e}")
                fundamentals = {}

        if export_csv:
            data.to_csv(csv_file)
            print(f'Data exported to {csv_file}')

    else:
        print("Data download failed or returned an empty DataFrame.")
        fundamentals = {}

    # Return a dictionary containing both market data and, if requested, fundamentals
    return {'market_data': data, 'fundamentals': fundamentals if include_fundamentals else None}

In [100]:
# Initialize an empty dictionary to store data for each fund
funds_data = {}

# Iterate over the index funds to fetch data for each and store it in the dictionary
for fund in index_funds:
    print(f"Fetching data for {fund}...")
    # Note: Adjust the parameters according to your fetch_financial_data function's definition
    # Here it's assumed that fetch_financial_data only requires the ticker symbol as a parameter
    funds_data[fund] = fetch_financial_data(ticker=fund, calculate_indicators=True)

# Now, extract the data for each fund into its own variable
SPY_data = funds_data['SPY']
QQQ_data = funds_data['QQQ']
DAX_data = funds_data.get('DAX')  # Using .get() for 'DAX' in case it's not available/fetched correctly


Fetching data for SPY...
Fetching data for QQQ...
Fetching data for DAX...


In [112]:
# Initialize an empty dictionary to store data for each ETF
etfs_data = {}

# Iterate over the leveraged ETFs to fetch data for each and store it in the dictionary
for etf in leveraged_etfs:
    print(f"Fetching data for {etf}...")
    # Adjust parameters as per your fetch_financial_data function's requirements
    etfs_data[etf] = fetch_financial_data(ticker=etf, calculate_indicators=True)
# Now, let's assume you want to access the data specifically for TQQQ, UDOW, and SOXL
# etfs_data
# TQQQ_data = etfs_data['TQQQ']
# UMDD_data = etfs_data['UMDD']
# UDOW_data = etfs_data['UDOW']
# SOXL_data = etfs_data['SOXL']
# NVDL_data = etfs_data['NVDL']
# TSLL_data = etfs_data['TSLL']
# BITX_data = etfs_data['BITX']


Fetching data for TQQQ...
Fetching data for UMDD...
Fetching data for UDOW...
Fetching data for SOXL...
Fetching data for NVDL...
Fetching data for TSLL...


In [117]:
monthly_dividend_data = {}

# Iterate over the monthly dividend REITs and ETFs to fetch data for each and store it in the dictionary
for asset in monthly_dividend_reits_etfs:
    print(f"Fetching data for {asset}...")
    # Adjust parameters as per your fetch_financial_data function's requirements
    monthly_dividend_data[asset] = fetch_financial_data(ticker=asset, calculate_indicators=True, include_fundamentals=True)

# Access the data specifically for each asset
# monthly_dividend_data
# monthly_dividend_data['O']
# Access the market data for 'O'
o_market_data = monthly_dividend_data['O']['market_data']

# Display the first few rows of the market data to see all columns

# If you have fetched fundamental data as well and want to display, for example, the 'info' part:
o_fundamental_info = monthly_dividend_data['O']['fundamentals']['info']


Fetching data for O...
Data exported to O_1d_data_1993-01-01_to_2024-03-19.csv
Fetching data for AGNC...
Data exported to AGNC_1d_data_1993-01-01_to_2024-03-19.csv
Fetching data for CSHI...
Data exported to CSHI_1d_data_1993-01-01_to_2024-03-19.csv
Fetching data for JEPI...
Data exported to JEPI_1d_data_1993-01-01_to_2024-03-19.csv
Fetching data for NUSI...
Data exported to NUSI_1d_data_1993-01-01_to_2024-03-19.csv


In [115]:
print("Market Data for 'O':")
print(o_market_data.head())


Market Data for 'O':
                Open      High       Low     Close  Adj Close  Volume  \
Date                                                                    
1994-11-14  7.509690  7.691376  7.388566  7.630814   1.271571   65842   
1994-11-15  7.449128  7.630814  7.388566  7.449128   1.241294   70382   
1994-11-16  7.449128  7.570252  7.388566  7.509690   1.251386   85862   
1994-11-17  7.388566  7.570252  7.388566  7.570252   1.261477   99485   
1994-11-18  7.509690  7.509690  7.388566  7.388566   1.231203  113933   

            Bollinger_High  Bollinger_Low      MACD    Signal        RSI  \
Date                                                                       
1994-11-14        8.543828       7.450600 -0.065216 -0.003688  37.488927   
1994-11-15        8.572326       7.391821 -0.088709 -0.020692  31.505253   
1994-11-16        8.525431       7.347873 -0.101273 -0.036808  35.217071   
1994-11-17        8.442942       7.339519 -0.105131 -0.050473  38.789343   
1994-11-18 

In [116]:

# Display fundamental 'info' data. Note: 'info' is a dictionary, so we format it for better readability
print("\nFundamental Info for 'O':")
for key, value in o_fundamental_info.items():
    print(f"{key}: {value}")


Fundamental Info for 'O':
address1: 11995 El Camino Real
city: San Diego
state: CA
zip: 92130
country: United States
phone: (858) 284-5000
fax: (858) 481-4860
website: https://www.realtyincome.com
industry: REIT - Retail
industryKey: reit-retail
industryDisp: REIT - Retail
sector: Real Estate
sectorKey: real-estate
sectorDisp: Real Estate
longBusinessSummary: Realty Income, The Monthly Dividend Company, is an S&P 500 company and member of the S&P 500 Dividend Aristocrats index. We invest in people and places to deliver dependable monthly dividends that increase over time. The company is structured as a real estate investment trust ("REIT"), and its monthly dividends are supported by the cash flow from over 15,450 real estate properties (including properties acquired in the Spirit merger in January 2024) primarily owned under long-term net lease agreements with commercial clients. To date, the company has declared 644 consecutive monthly dividends on its shares of common stock througho

In [ ]:
# Display fundamental 'info' data. Note: 'info' is a dictionary, so we format it for better readability
print("\nFundamental Info for 'O':")
for key, value in o_fundamental_info.items():
    print(f"{key}: {value}")

In [ ]:
print("Market Data for 'O':")
print(o_market_data.head())
# Display fundamental 'info' data. Note: 'info' is a dictionary, so we format it for better readability
print("\nFundamental Info for 'O':")
for key, value in o_fundamental_info.items():
    print(f"{key}: {value}")

In [103]:
# Initialize an empty dictionary to store data for each stock
quarterly_dividend_data = {}

# Iterate over the quarterly dividend stocks to fetch data for each and store it in the dictionary
for stock in quarterly_dividend_stocks:
    print(f"Fetching data for {stock}...")
    quarterly_dividend_data[stock] = fetch_financial_data(ticker=stock, calculate_indicators=True)

# Now, you can access the data specifically for each stock, for example:
# MSFT_data = quarterly_dividend_data['MSFT']
# quarterly_dividend_data
# MSFT_data

Fetching data for SPYD...
Fetching data for MSFT...
Fetching data for INTC...
Fetching data for F...
Fetching data for CSCO...
Fetching data for BAC...
Fetching data for PFE...
Fetching data for BX...
Fetching data for MO...
Fetching data for DOW...
Fetching data for WMT...
Fetching data for T...
Fetching data for KMB...
Fetching data for SWK...
Fetching data for IBM...
Fetching data for PEP...
Fetching data for KO...
Fetching data for JNJ...


# Beta Weight

In [104]:
def calculate_beta(stock_symbol, market_symbol='^GSPC', start='2020-01-01', end='2023-01-01'):
    try:
        stock_data = yf.download(stock_symbol, start=start, end=end)['Adj Close']
        market_data = yf.download(market_symbol, start=start, end=end)['Adj Close']
    except Exception as e:
        return {'error': f'Failed to download data: {str(e)}'}
    
    returns = pd.DataFrame({
        'stock_returns': stock_data.pct_change(),
        'market_returns': market_data.pct_change()
    }).dropna()
    
    betas = {}
    
    # Method 1: Linear Regression
    try:
        model = LinearRegression().fit(returns[['market_returns']], returns['stock_returns'])
        betas['linear_regression'] = model.coef_[0]
    except Exception as e:
        betas['linear_regression_error'] = str(e)
    
    # Method 2: Covariance Method
    try:
        covariance = returns.cov().iloc[0, 1]
        market_var = returns['market_returns'].var()
        betas['covariance_method'] = covariance / market_var
    except Exception as e:
        betas['covariance_method_error'] = str(e)
    
    # Method 3: Variance Ratio
    try:
        stock_var = returns['stock_returns'].var()
        betas['variance_ratio'] = stock_var / market_var
    except Exception as e:
        betas['variance_ratio_error'] = str(e)
    
    # Method 4: Using scipy linregress for an alternative linear regression method
    try:
        slope, _, _, _, _ = linregress(returns['market_returns'], returns['stock_returns'])
        betas['linregress'] = slope
    except Exception as e:
        betas['linregress_error'] = str(e)
    
    return betas

In [105]:
# Combine all asset lists into a single dictionary for easier iteration
assets = {
    'Index Funds': index_funds,
    'LETFS': leveraged_etfs,
    'Monthly Dividend REITs/ETFs': monthly_dividend_reits_etfs,
    'Quarterly Dividend Stocks': quarterly_dividend_stocks
}

# Initialize a dictionary to store beta values for each asset category
beta_values = {category: {} for category in assets.keys()}

# Iterate through each category and asset to calculate beta values
for category, asset_list in assets.items():
    print(f"\nCalculating beta for {category}:")
    for asset in asset_list:
        try:
            beta = calculate_beta(asset)
            print(f"  {asset}:")
            for method, value in beta.items():
                print(f"    {method}: {value}")
        except Exception as e:
            print(f"  Error calculating beta for {asset}: {str(e)}")
        beta_values[category][asset] = beta


Calculating beta for Index Funds:


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

  SPY:
    linear_regression: 0.980756700346283
    covariance_method: 0.9807567003462827
    variance_ratio: 0.9640118642315548
    linregress: 0.9807567003462829



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


  QQQ:
    linear_regression: 1.0864858935578845
    covariance_method: 1.086485893557884
    variance_ratio: 1.3591378803497065
    linregress: 1.0864858935578843


...

  PEP:
    linear_regression: 0.7336126048118109
    covariance_method: 0.7336126048118101
    variance_ratio: 0.9933024201017637
    linregress: 0.7336126048118102



[*********************100%%**********************]  1 of 1 completed


  KO:
    linear_regression: 0.686321707257074
    covariance_method: 0.6863217072570734
    variance_ratio: 0.934749318106406
    linregress: 0.6863217072570735


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

  JNJ:
    linear_regression: 0.5395766963969222
    covariance_method: 0.5395766963969215
    variance_ratio: 0.7351044430886029
    linregress: 0.5395766963969216


In [106]:
beta_values

{'Index Funds': {'SPY': {'linear_regression': 0.980756700346283,
   'covariance_method': 0.9807567003462827,
   'variance_ratio': 0.9640118642315548,
   'linregress': 0.9807567003462829},
  'QQQ': {'linear_regression': 1.0864858935578845,
   'covariance_method': 1.086485893557884,
   'variance_ratio': 1.3591378803497065,
   'linregress': 1.0864858935578843},
  'DAX': {'linear_regression': 0.8825338592042832,
   'covariance_method': 0.8825338592042831,
   'variance_ratio': 1.2270079681820172,
   'linregress': 0.8825338592042833}},
 'LETFS': {'TQQQ': {'linear_regression': 3.2001769060090437,
   'covariance_method': 3.200176906009043,
   'variance_ratio': 11.845921384123255,
   'linregress': 3.200176906009043},
  'UMDD': {'linear_regression': 3.130596898398864,
   'covariance_method': 3.1305968983988652,
   'variance_ratio': 11.681723384549286,
   'linregress': 3.1305968983988652},
  'UDOW': {'linear_regression': 2.7727876301817016,
   'covariance_method': 2.772787630181703,
   'variance_

# Valuation Models for Each Asset Class:

In [55]:
def calculate_cashflow_growth_rate(free_cash_flows):
    return free_cash_flows.pct_change().mean()

def project_future_free_cash_flows(last_cash_flow, growth_rate, years):
    return [last_cash_flow * (1 + growth_rate) ** i for i in range(1, years + 1)]


def calculate_terminal_value(last_cash_flow, growth_rate, required_rate, years):
    return last_cash_flow * (1 + growth_rate) / (required_rate - growth_rate) / (1 + required_rate) ** years

def calculate_fair_value(discounted_cash_flows, terminal_value, outstanding_shares):
    total_present_value = sum(discounted_cash_flows) + terminal_value
    return total_present_value / outstanding_shares

def get_cost_of_equity(risk_free_rate, beta, market_return):
    return risk_free_rate + beta * (market_return - risk_free_rate)

def get_cost_of_debt(interest_rate, tax_rate):
    return interest_rate * (1 - tax_rate)

def get_proportions(market_value_equity, market_value_debt):
    total_value = market_value_equity + market_value_debt
    return market_value_equity / total_value, market_value_debt / total_value

def calculate_wacc(cost_of_equity, cost_of_debt, equity_proportion, debt_proportion, tax_rate):
    wacc = (cost_of_equity * equity_proportion) + ((1 - tax_rate) * cost_of_debt * debt_proportion)
    return wacc

def calculate_intrinsic_value(dividend_data, discount_rate):
    intrinsic_value = 0
    for year, dividend in enumerate(dividend_data, start=1):
        if year <= 5:
            growth_rate = 0.05
        elif 5 < year <= 10:
            growth_rate = 0.03
        else:
            growth_rate = 0.01
        intrinsic_value += dividend / ((1 + discount_rate) ** year)
    return intrinsic_value

def calculate_cost_of_equity(beta, risk_free_rate, market_return):
    """
    Calculate the cost of equity using the CAPM formula.
    
    :param beta: Beta of the stock
    :param risk_free_rate: Risk-free rate
    :param market_return: Expected market return
    :return: Cost of equity
    """
    return risk_free_rate + beta * (market_return - risk_free_rate)


In [57]:
def dcf_valuation(cash_flows, discount_rate):
    """
    Calculate the present value of cash flows using the discounted cash flow (DCF) method.
    
    Args:
    - cash_flows (list): List of projected cash flows.
    - discount_rate (float): Discount rate (required rate of return).
    
    Returns:
    - float: Present value of the cash flows.
    """
    dcf_value = sum(cf / (1 + discount_rate)**n for n, cf in enumerate(cash_flows, start=1))
    return dcf_value

def calculate_expected_return(risk_free_rate, beta, market_return, market_risk_premium):
    """
    Calculate the expected return of an asset using the Capital Asset Pricing Model (CAPM).
    
    Args:
    - risk_free_rate (float): Risk-free rate (e.g., yield on Treasury bills).
    - beta (float): Beta coefficient of the asset.
    - market_return (float): Expected return of the market portfolio.
    - market_risk_premium (float): Market risk premium.
    
    Returns:
    - float: Expected return of the asset.
    """
    expected_return = risk_free_rate + beta * market_risk_premium
    return expected_return

def three_stage_dividend_discount_model(symbol, discount_rate):
    dividend_data = fetch_dividend_data(symbol)
    intrinsic_value = calculate_intrinsic_value(dividend_data, discount_rate)
    return intrinsic_value


def residual_income_model(net_income, equity, required_return):
    """
    Calculate the value of equity using the Residual Income Model.
    
    Args:
    - net_income (float): Net income of the company.
    - equity (float): Book value of equity.
    - required_return (float): Required rate of return on equity.
    
    Returns:
    - float: Estimated value of equity using the Residual Income Model.
    """
    # Calculate the present value of expected future residual income
    residual_income = net_income - (required_return * equity)
    
    # Value of equity is the book value of equity plus the present value of expected future residual income
    equity_value = equity + residual_income
    
    return equity_value



# Backtesting for DCA Strategy

# Options Pricing Analysis